In [1]:
import pandas as pd

# pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Encoders
from sklearn.preprocessing import OneHotEncoder


# Metrics
from sklearn.metrics import classification_report